# Wczytywanie danych

In [71]:
import torch
from torch_geometric.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'device')

# Wczytaj zbiór uczący
train_dataset = torch.load('train_dataset.pt')
# wczytaj zbiór testowy
test_dataset = torch.load('test_dataset.pt')

print(len(train_dataset))
print(len(test_dataset))

# Definiuj DataLoader dla zbiorów uczącego i testowego
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

184
46


# ChGW0

In [69]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv

class GraphRegressionModel(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, dropout=0.5, momentum=0.5):
        super(GraphRegressionModel, self).__init__()
        name = 'ChGW0'

        # Graph Convolutional Layer
        self.conv1 = GraphConv(num_node_features, hidden_dim)

        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(hidden_dim + 2, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

        # Store momentum as an attribute
        self.momentum = momentum
        
    def forward(self, data):
        x, edge_index, edge_weight, global_feature = data.x, data.edge_index, data.edge_weight, data.global_feature

        # Apply Graph Convolution
        x = self.conv1(x, edge_index, edge_weight=edge_weight)

        # Global features concatenation
        global_feature = global_feature.expand(x.size(0), -1)  # Dostosuj global_feature do rozmiaru x
        x = torch.cat([x, global_feature], dim=1)

        # Fully Connected Layers with Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x
    
    def loss(self, pred, score):
        return F.mse_loss(pred, score)

# testy

In [70]:
import torch
import torch.optim as optim
from torch_geometric.data import DataLoader
from sklearn.model_selection import KFold
from tqdm import tqdm

# Import your GraphRegressionModel here...

device = torch.device('cuda' if torch.cuda.is_available() else 'device')

# Load your training and test datasets
train_dataset = torch.load('train_dataset.pt')
test_dataset = torch.load('test_dataset.pt')

# Combine the datasets into a single list for cross-validation
all_datasets = train_dataset + test_dataset

# Convert the combined datasets to PyTorch DataLoader for batch training
dataloader = DataLoader(all_datasets, batch_size=1, shuffle=True)

# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Define hyperparameters
num_epochs = 50
learning_rate = 0.001

# Training loop for cross-validation
for fold, (train_idx, val_idx) in enumerate(kf.split(all_datasets)):
    print(f"Fold {fold + 1}/{num_folds}")

    # Split the data into training and validation sets
    train_data = [all_datasets[i] for i in train_idx]
    val_data = [all_datasets[i] for i in val_idx]

    # Initialize a new model for each fold
    model = GraphRegressionModel().to(device)

    # Set the model to training mode
    model.train()

    # Initialize the optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in tqdm(range(num_epochs), desc="Epoch"):
        for data in train_data:
            # Zero the gradients
            optimizer.zero_grad()

            # Send data to the device
            data = data.to(device)

            # Forward pass
            output = model(data)

            # Calculate the loss
            loss = model.loss(output, data.y.view(-1, 1).float())

            # Backward pass
            loss.backward()

            # Update weights
            optimizer.step()

    # Set the model to evaluation mode
    model.eval()

    # Validation loop
    val_losses = []
    with torch.no_grad():
        for data in val_data:
            # Send data to the device
            data = data.to(device)

            output = model(data)
            val_loss = model.loss(output, data.y.view(-1, 1).float())
            val_losses.append(val_loss.item())

    # Calculate and print the average validation loss for the fold
    avg_val_loss = sum(val_losses) / len(val_losses)
    print(f"Validation Loss: {avg_val_loss}")


Fold 1/10


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]/tmp/ipykernel_24189/2804048419.py:40: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred, score)
Epoch: 100%|██████████| 50/50 [00:09<00:00,  5.30it/s]


Validation Loss: 56229.4624691165
Fold 2/10


Epoch: 100%|██████████| 50/50 [00:10<00:00,  4.96it/s]


Validation Loss: 50692.229821246605
Fold 3/10


Epoch:  96%|█████████▌| 48/50 [00:09<00:00,  5.02it/s]


KeyboardInterrupt: 

# Testy2

In [ ]:
from sklearn.metrics import r2_score
import torch
import torch.optim as optim
from torch_geometric.data import DataLoader
from torch_geometric.data import Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import numpy as np



# Define the hyperparameter values to test
hidden_dim_values = list(range(16, 512, 16))
dropout_values = np.arange(0.05, 1.0, 0.05)
momentum_values = np.arange(0.05, 1.0, 0.05)

# Initialize a dictionary to store results
part_results ={}
results = {}

# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Loop over hyperparameters
for hidden_dim in hidden_dim_values:
    for dropout in dropout_values:
        for momentum in momentum_values:
            print(f"\nTesting for hidden_dim={hidden_dim}, dropout={dropout}, momentum={momentum}")

            # Initialize a dictionary to store results for the current hyperparameters
            hyperparam_results = {'MSE': [], 'R2': []}

            # Loop over folds
            for fold, (train_idx, val_idx) in enumerate(kf.split(train_dataset)):
                train_data = [train_dataset[i] for i in train_idx]
                val_data = [train_dataset[i] for i in val_idx]

                model = GraphRegressionModel(num_node_features=1, hidden_dim=hidden_dim, output_dim=1, dropout=dropout, momentum=momentum)
                optimizer = optim.Adam(model.parameters(), lr=0.001)

                # Training loop
                model.train()
                for epoch in tqdm(range(10), desc=f"Fold {fold + 1}/{num_folds}"):
                    for data in train_data:
                        optimizer.zero_grad()
                        output = model(data)
                        loss = model.loss(output, data.y.view(-1, 1).float())
                        loss.backward()
                        optimizer.step()

                # Validation loop
                model.eval()
                y_true = []
                y_pred = []
                
                for data in val_data:
                    with torch.no_grad():    
                        output = model(data)[0][0].item()                
                        y_pred.append(output)

                    y_true.append(data.y.item())

                # Calculate and store Mean Squared Error (MSE)
                r2 = r2_score(y_true, y_pred)
                hyperparam_results['R2'].append(r2)
                mse = mean_squared_error(y_true, y_pred)
                hyperparam_results['MSE'].append(mse)
                part_results[(hidden_dim, dropout, momentum, fold)] = {'MSE': mse, 'R2' : r2}
                with  open('chkgw0_spec.txt', 'w') as file2:
                    file2.write('\n'+f"Hidden Dim: {hidden_dim}, Dropout: {dropout}, Momentum: {momentum}, Fold: {fold}, MSE: {mse}, R2: {r2}", file=file2)

            # Calculate average MSE across folds
            avg_mse = sum(hyperparam_results['MSE']) / num_folds
            avg_r2 = sum(hyperparam_results['R2']) / num_folds
            results[(hidden_dim, dropout, momentum)] = {'Avg MSE': avg_mse, 'Avg R2': avg_r2}
            # Open files for writing
            with open('chkgw0_avg.txt', 'w') as file1:
                # Print the results to file1
                     file1.write('\n'+"Hidden Dim: {hidden_dim}, Dropout: {dropout}, Momentum: {momentum}, Avg MSE: {avg_mse}, Avg R2: {avg_r2}", file=file1)





# Model podstawowy, ale bez dropout'u - chyba najlepszy

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv

class GraphRegressionModel(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, momentum=0.5):
        super(GraphRegressionModel, self).__init__()

        # Graph Convolutional Layer
        self.conv1 = GraphConv(num_node_features, hidden_dim)

        # Fully Connected Layers (without Dropout)
        self.fc1 = nn.Linear(hidden_dim + 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        # Store momentum as an attribute
        self.momentum = momentum
    def forward(self, data):
        x, edge_index, edge_weight, global_feature = data.x, data.edge_index, data.edge_weight, data.global_feature

        # Apply Graph Convolution
        x = self.conv1(x, edge_index, edge_weight=edge_weight)

        # Global features concatenation
        global_feature = global_feature.expand(x.size(0), -1)
        x = torch.cat([x, global_feature], dim=1)

        # Fully Connected Layers (without Dropout)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x
    
    def loss(self, pred, score):
        return F.mse_loss(pred, score)


In [ ]:
from sklearn.metrics import r2_score
import torch
import torch.optim as optim
from torch_geometric.data import DataLoader
from torch_geometric.data import Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import numpy as np



# Define the hyperparameter values to test
hidden_dim_values = list(range(16, 512, 16))
momentum_values = np.arange(0.05, 1.0, 0.05)

# Initialize a dictionary to store results
part_results ={}
results = {}

# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Loop over hyperparameters
for hidden_dim in hidden_dim_values:
        for momentum in momentum_values:
            print(f"\nTesting for hidden_dim={hidden_dim}, momentum={momentum}")

            # Initialize a dictionary to store results for the current hyperparameters
            hyperparam_results = {'MSE': [], 'R2': []}

            # Loop over folds
            for fold, (train_idx, val_idx) in enumerate(kf.split(train_dataset)):
                train_data = [train_dataset[i] for i in train_idx]
                val_data = [train_dataset[i] for i in val_idx]

                model = GraphRegressionModel(num_node_features=1, hidden_dim=hidden_dim, output_dim=1, momentum=momentum)
                optimizer = optim.Adam(model.parameters(), lr=0.001)

                # Training loop
                model.train()
                for epoch in tqdm(range(10), desc=f"Fold {fold + 1}/{num_folds}"):
                    for data in train_data:
                        optimizer.zero_grad()
                        output = model(data)
                        loss = model.loss(output, data.y.view(-1, 1).float())
                        loss.backward()
                        optimizer.step()

                # Validation loop
                model.eval()
                y_true = []
                y_pred = []
                
                for data in val_data:
                    with torch.no_grad():    
                        output = model(data)[0][0].item()                
                        y_pred.append(output)

                    y_true.append(data.y.item())

                # Calculate and store Mean Squared Error (MSE)
                r2 = r2_score(y_true, y_pred)
                hyperparam_results['R2'].append(r2)
                mse = mean_squared_error(y_true, y_pred)
                hyperparam_results['MSE'].append(mse)
                part_results[(hidden_dim, momentum, fold)] = {'MSE': mse, 'R2' : r2}

            # Calculate average MSE across folds
            avg_mse = sum(hyperparam_results['MSE']) / num_folds
            avg_r2 = sum(hyperparam_results['R2']) / num_folds
            results[(hidden_dim, momentum)] = {'Avg MSE': avg_mse, 'Avg R2': avg_r2}


# Open files for writing
with open('noDrop_avg.txt', 'w') as file1, open('noDrop_spec.txt', 'w') as file2:
    # Print the results to file1
    for (hidden_dim, dropout), metrics in results.items():
        print(f"Hidden Dim: {hidden_dim}, Momentum: {momentum}, Avg MSE: {metrics['Avg MSE']}, Avg R2: {metrics['Avg R2']}", file=file1)

    # Print the partial results to file2
    for (hidden_dim, dropout, fold), metrics in part_results.items():
        print(f"Hidden Dim: {hidden_dim}, Momentum: {momentum}, Fold: {fold}, MSE: {metrics['MSE']}, R2: {metrics['R2']}", file=file2)



# Model podstawowy bez cech globalnych

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv

class GraphRegressionModel(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, dropout=0.5, momentum=0.5):
        super(GraphRegressionModel, self).__init__()

        # Graph Convolutional Layer
        self.conv1 = GraphConv(num_node_features, hidden_dim)

        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        # Store momentum as an attribute
        self.momentum = momentum
    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_weight

        # Apply Graph Convolution
        x = self.conv1(x, edge_index, edge_weight=edge_weight)

        # Fully Connected Layers with Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x
    
    def loss(self, pred, score):
        return F.mse_loss(pred, score)


In [ ]:
from sklearn.metrics import r2_score
import torch
import torch.optim as optim
from torch_geometric.data import DataLoader
from torch_geometric.data import Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import numpy as np



# Define the hyperparameter values to test
hidden_dim_values = list(range(16, 512, 16))
dropout_values = np.arange(0.05, 1.0, 0.05)
momentum_values = np.arange(0.05, 1.0, 0.05)

# Initialize a dictionary to store results
part_results ={}
results = {}

# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Loop over hyperparameters
for hidden_dim in hidden_dim_values:
    for dropout in dropout_values:
        for momentum in momentum_values:
            print(f"\nTesting for hidden_dim={hidden_dim}, dropout={dropout}, momentum={momentum}")

            # Initialize a dictionary to store results for the current hyperparameters
            hyperparam_results = {'MSE': [], 'R2': []}

            # Loop over folds
            for fold, (train_idx, val_idx) in enumerate(kf.split(train_dataset)):
                train_data = [train_dataset[i] for i in train_idx]
                val_data = [train_dataset[i] for i in val_idx]

                model = GraphRegressionModel(num_node_features=1, hidden_dim=hidden_dim, output_dim=1, dropout=dropout, momentum=momentum)
                optimizer = optim.Adam(model.parameters(), lr=0.001)

                # Training loop
                model.train()
                for epoch in tqdm(range(10), desc=f"Fold {fold + 1}/{num_folds}"):
                    for data in train_data:
                        optimizer.zero_grad()
                        output = model(data)
                        loss = model.loss(output, data.y.view(-1, 1).float())
                        loss.backward()
                        optimizer.step()

                # Validation loop
                model.eval()
                y_true = []
                y_pred = []
                
                for data in val_data:
                    with torch.no_grad():    
                        output = model(data)[0][0].item()                
                        y_pred.append(output)

                    y_true.append(data.y.item())

                # Calculate and store Mean Squared Error (MSE)
                r2 = r2_score(y_true, y_pred)
                hyperparam_results['R2'].append(r2)
                mse = mean_squared_error(y_true, y_pred)
                hyperparam_results['MSE'].append(mse)
                part_results[(hidden_dim, dropout, momentum, fold)] = {'MSE': mse, 'R2' : r2}

            # Calculate average MSE across folds
            avg_mse = sum(hyperparam_results['MSE']) / num_folds
            avg_r2 = sum(hyperparam_results['R2']) / num_folds
            results[(hidden_dim, dropout, momentum)] = {'Avg MSE': avg_mse, 'Avg R2': avg_r2}


# Open files for writing
with open('noGlob_avg.txt', 'w') as file1, open('noGlob_spec.txt', 'w') as file2:
    # Print the results to file1
    for (hidden_dim, dropout), metrics in results.items():
        print(f"Hidden Dim: {hidden_dim}, Dropout: {dropout}, Momentum: {momentum}, Avg MSE: {metrics['Avg MSE']}, Avg R2: {metrics['Avg R2']}", file=file1)

    # Print the partial results to file2
    for (hidden_dim, dropout, fold), metrics in part_results.items():
        print(f"Hidden Dim: {hidden_dim}, Dropout: {dropout}, Momentum: {momentum}, Fold: {fold}, MSE: {metrics['MSE']}, R2: {metrics['R2']}", file=file2)



# ChGW1

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GraphRegressionModelGCN(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, dropout=0.5, momentum=0.5):
        super(GraphRegressionModelGCN, self).__init__()

        # Graph Convolutional Layer using GCNConv
        self.conv1 = GCNConv(num_node_features, hidden_dim)

        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(hidden_dim + 2, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        # Store momentum as an attribute
        self.momentum = momentum
    def forward(self, data):
        x, edge_index, edge_weight, global_feature = data.x, data.edge_index, data.edge_weight, data.global_feature

        # Apply Graph Convolution using GCNConv
        x = self.conv1(x, edge_index, edge_weight=edge_weight)

        # Global features concatenation
        global_feature = global_feature.expand(x.size(0), -1)
        x = torch.cat([x, global_feature], dim=1)

        # Fully Connected Layers with Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x
    
    def loss(self, pred, score):
        return F.mse_loss(pred, score)

In [ ]:
from sklearn.metrics import r2_score
import torch
import torch.optim as optim
from torch_geometric.data import DataLoader
from torch_geometric.data import Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import numpy as np



# Define the hyperparameter values to test
hidden_dim_values = list(range(16, 512, 16))
dropout_values = np.arange(0.05, 1.0, 0.05)
momentum_values = np.arange(0.05, 1.0, 0.05)

# Initialize a dictionary to store results
part_results ={}
results = {}

# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Loop over hyperparameters
for hidden_dim in hidden_dim_values:
    for dropout in dropout_values:
        for momentum in momentum_values:
            print(f"\nTesting for hidden_dim={hidden_dim}, dropout={dropout}, momentum={momentum}")

            # Initialize a dictionary to store results for the current hyperparameters
            hyperparam_results = {'MSE': [], 'R2': []}

            # Loop over folds
            for fold, (train_idx, val_idx) in enumerate(kf.split(train_dataset)):
                train_data = [train_dataset[i] for i in train_idx]
                val_data = [train_dataset[i] for i in val_idx]

                model = GraphRegressionModel(num_node_features=1, hidden_dim=hidden_dim, output_dim=1, dropout=dropout, momentum=momentum)
                optimizer = optim.Adam(model.parameters(), lr=0.001)

                # Training loop
                model.train()
                for epoch in tqdm(range(10), desc=f"Fold {fold + 1}/{num_folds}"):
                    for data in train_data:
                        optimizer.zero_grad()
                        output = model(data)
                        loss = model.loss(output, data.y.view(-1, 1).float())
                        loss.backward()
                        optimizer.step()

                # Validation loop
                model.eval()
                y_true = []
                y_pred = []
                
                for data in val_data:
                    with torch.no_grad():    
                        output = model(data)[0][0].item()                
                        y_pred.append(output)

                    y_true.append(data.y.item())

                # Calculate and store Mean Squared Error (MSE)
                r2 = r2_score(y_true, y_pred)
                hyperparam_results['R2'].append(r2)
                mse = mean_squared_error(y_true, y_pred)
                hyperparam_results['MSE'].append(mse)
                part_results[(hidden_dim, dropout, momentum, fold)] = {'MSE': mse, 'R2' : r2}

            # Calculate average MSE across folds
            avg_mse = sum(hyperparam_results['MSE']) / num_folds
            avg_r2 = sum(hyperparam_results['R2']) / num_folds
            results[(hidden_dim, dropout, momentum)] = {'Avg MSE': avg_mse, 'Avg R2': avg_r2}


# Open files for writing
with open('chkgw1_avg.txt', 'w') as file1, open('chkgw1_spec.txt', 'w') as file2:
    # Print the results to file1
    for (hidden_dim, dropout), metrics in results.items():
        print(f"Hidden Dim: {hidden_dim}, Dropout: {dropout}, Momentum: {momentum}, Avg MSE: {metrics['Avg MSE']}, Avg R2: {metrics['Avg R2']}", file=file1)

    # Print the partial results to file2
    for (hidden_dim, dropout, fold), metrics in part_results.items():
        print(f"Hidden Dim: {hidden_dim}, Dropout: {dropout}, Momentum: {momentum}, Fold: {fold}, MSE: {metrics['MSE']}, R2: {metrics['R2']}", file=file2)



# ChGW2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import ChebConv

class GraphRegressionModelCheb(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, dropout=0.5, momentum=0.5):
        super(GraphRegressionModelCheb, self).__init__()

        # Chebyshev Graph Convolutional Layer
        self.conv1 = ChebConv(num_node_features, hidden_dim, K=2)  # K is the filter size (you can adjust it)

        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(hidden_dim + 2, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        # Store momentum as an attribute
        self.momentum = momentum
    def forward(self, data):
        x, edge_index, edge_weight, global_feature = data.x, data.edge_index, data.edge_weight, data.global_feature

        # Apply Chebyshev Graph Convolution
        x = self.conv1(x, edge_index, edge_weight=edge_weight)

        # Global features concatenation
        global_feature = global_feature.expand(x.size(0), -1)
        x = torch.cat([x, global_feature], dim=1)

        # Fully Connected Layers with Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x
    
    def loss(self, pred, score):
        return F.mse_loss(pred, score)

In [ ]:
from sklearn.metrics import r2_score
import torch
import torch.optim as optim
from torch_geometric.data import DataLoader
from torch_geometric.data import Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import numpy as np



# Define the hyperparameter values to test
hidden_dim_values = list(range(16, 512, 16))
dropout_values = np.arange(0.05, 1.0, 0.05)
momentum_values = np.arange(0.05, 1.0, 0.05)

# Initialize a dictionary to store results
part_results ={}
results = {}

# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Loop over hyperparameters
for hidden_dim in hidden_dim_values:
    for dropout in dropout_values:
        for momentum in momentum_values:
            print(f"\nTesting for hidden_dim={hidden_dim}, dropout={dropout}, momentum={momentum}")

            # Initialize a dictionary to store results for the current hyperparameters
            hyperparam_results = {'MSE': [], 'R2': []}

            # Loop over folds
            for fold, (train_idx, val_idx) in enumerate(kf.split(train_dataset)):
                train_data = [train_dataset[i] for i in train_idx]
                val_data = [train_dataset[i] for i in val_idx]

                model = GraphRegressionModel(num_node_features=1, hidden_dim=hidden_dim, output_dim=1, dropout=dropout, momentum=momentum)
                optimizer = optim.Adam(model.parameters(), lr=0.001)

                # Training loop
                model.train()
                for epoch in tqdm(range(10), desc=f"Fold {fold + 1}/{num_folds}"):
                    for data in train_data:
                        optimizer.zero_grad()
                        output = model(data)
                        loss = model.loss(output, data.y.view(-1, 1).float())
                        loss.backward()
                        optimizer.step()

                # Validation loop
                model.eval()
                y_true = []
                y_pred = []
                
                for data in val_data:
                    with torch.no_grad():    
                        output = model(data)[0][0].item()                
                        y_pred.append(output)

                    y_true.append(data.y.item())

                # Calculate and store Mean Squared Error (MSE)
                r2 = r2_score(y_true, y_pred)
                hyperparam_results['R2'].append(r2)
                mse = mean_squared_error(y_true, y_pred)
                hyperparam_results['MSE'].append(mse)
                part_results[(hidden_dim, dropout, momentum, fold)] = {'MSE': mse, 'R2' : r2}

            # Calculate average MSE across folds
            avg_mse = sum(hyperparam_results['MSE']) / num_folds
            avg_r2 = sum(hyperparam_results['R2']) / num_folds
            results[(hidden_dim, dropout, momentum)] = {'Avg MSE': avg_mse, 'Avg R2': avg_r2}


# Open files for writing
with open('chkgw2_avg.txt', 'w') as file1, open('chkgw2_spec.txt', 'w') as file2:
    # Print the results to file1
    for (hidden_dim, dropout), metrics in results.items():
        print(f"Hidden Dim: {hidden_dim}, Dropout: {dropout}, Momentum: {momentum}, Avg MSE: {metrics['Avg MSE']}, Avg R2: {metrics['Avg R2']}", file=file1)

    # Print the partial results to file2
    for (hidden_dim, dropout, fold), metrics in part_results.items():
        print(f"Hidden Dim: {hidden_dim}, Dropout: {dropout}, Momentum: {momentum}, Fold: {fold}, MSE: {metrics['MSE']}, R2: {metrics['R2']}", file=file2)



# EdgeConv

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv, EdgeConv

class CustomEdgeConv(nn.Module):
    def __init__(self, num_node_features, hidden_dim):
        super(CustomEdgeConv, self).__init__()
        self.layers = nn.ModuleList([
            nn.Linear(1, 32),  # Adjust the input size for the first linear layer
            nn.ReLU(),
            nn.Linear(32, hidden_dim)
        ])

    def forward(self, x, edge_index):
        for layer in self.layers:
            x = layer(x)
        return x

class GraphRegressionModel(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, dropout=0.5, momentum=0.5):
        super(GraphRegressionModel, self).__init__()

        # Graph Convolutional Layer
        #self.conv1 = GraphConv(num_node_features, hidden_dim)

        # Edge Convolutional Layer
        self.conv2 = CustomEdgeConv(num_node_features, hidden_dim+2)

        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(hidden_dim + 4, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        # Store momentum as an attribute
        self.momentum = momentum
    def forward(self, data):
        x, edge_index, edge_weight, global_feature = data.x, data.edge_index, data.edge_weight, data.global_feature

        # Apply Graph Convolution
        #x = self.conv1(x, edge_index, edge_weight=edge_weight)
        # Apply Edge Convolution
        x = self.conv2(x, edge_index)

        # Global features concatenation
        global_feature = global_feature.expand(x.size(0), -1)
        x = torch.cat([x, global_feature], dim=1)

        # Fully Connected Layers with Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

    def loss(self, pred, score):
        return F.mse_loss(pred, score)

In [ ]:
from sklearn.metrics import r2_score
import torch
import torch.optim as optim
from torch_geometric.data import DataLoader
from torch_geometric.data import Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import numpy as np



# Define the hyperparameter values to test
hidden_dim_values = list(range(16, 512, 16))
dropout_values = np.arange(0.05, 1.0, 0.05)
momentum_values = np.arange(0.05, 1.0, 0.05)

# Initialize a dictionary to store results
part_results ={}
results = {}

# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Loop over hyperparameters
for hidden_dim in hidden_dim_values:
    for dropout in dropout_values:
        for momentum in momentum_values:
            print(f"\nTesting for hidden_dim={hidden_dim}, dropout={dropout}, momentum={momentum}")

            # Initialize a dictionary to store results for the current hyperparameters
            hyperparam_results = {'MSE': [], 'R2': []}

            # Loop over folds
            for fold, (train_idx, val_idx) in enumerate(kf.split(train_dataset)):
                train_data = [train_dataset[i] for i in train_idx]
                val_data = [train_dataset[i] for i in val_idx]

                model = GraphRegressionModel(num_node_features=1, hidden_dim=hidden_dim, output_dim=1, dropout=dropout, momentum=momentum)
                optimizer = optim.Adam(model.parameters(), lr=0.001)

                # Training loop
                model.train()
                for epoch in tqdm(range(10), desc=f"Fold {fold + 1}/{num_folds}"):
                    for data in train_data:
                        optimizer.zero_grad()
                        output = model(data)
                        loss = model.loss(output, data.y.view(-1, 1).float())
                        loss.backward()
                        optimizer.step()

                # Validation loop
                model.eval()
                y_true = []
                y_pred = []
                
                for data in val_data:
                    with torch.no_grad():    
                        output = model(data)[0][0].item()                
                        y_pred.append(output)

                    y_true.append(data.y.item())

                # Calculate and store Mean Squared Error (MSE)
                r2 = r2_score(y_true, y_pred)
                hyperparam_results['R2'].append(r2)
                mse = mean_squared_error(y_true, y_pred)
                hyperparam_results['MSE'].append(mse)
                part_results[(hidden_dim, dropout, momentum, fold)] = {'MSE': mse, 'R2' : r2}

            # Calculate average MSE across folds
            avg_mse = sum(hyperparam_results['MSE']) / num_folds
            avg_r2 = sum(hyperparam_results['R2']) / num_folds
            results[(hidden_dim, dropout, momentum)] = {'Avg MSE': avg_mse, 'Avg R2': avg_r2}


# Open files for writing
with open('edgeConv_avg.txt', 'w') as file1, open('edgeConv_spec.txt', 'w') as file2:
    # Print the results to file1
    for (hidden_dim, dropout), metrics in results.items():
        print(f"Hidden Dim: {hidden_dim}, Dropout: {dropout}, Momentum: {momentum}, Avg MSE: {metrics['Avg MSE']}, Avg R2: {metrics['Avg R2']}", file=file1)

    # Print the partial results to file2
    for (hidden_dim, dropout, fold), metrics in part_results.items():
        print(f"Hidden Dim: {hidden_dim}, Dropout: {dropout}, Momentum: {momentum}, Fold: {fold}, MSE: {metrics['MSE']}, R2: {metrics['R2']}", file=file2)



# GCNConv

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
    

class GraphRegressionModel(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, dropout=0.5, momentum=0.5):
        super(GraphRegressionModel, self).__init__()

        # Graph Convolutional Layer
        self.conv1 = GCNConv(num_node_features, hidden_dim)


        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(hidden_dim + 2, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

        # Store momentum as an attribute
        self.momentum = momentum
    def forward(self, data):
        x, edge_index, edge_weight, global_feature = data.x, data.edge_index, data.edge_weight, data.global_feature

        # Apply Graph Convolution
        x = self.conv1(x, edge_index, edge_weight=edge_weight)


        # Global features concatenation
        global_feature = global_feature.expand(x.size(0), -1)
        x = torch.cat([x, global_feature], dim=1)

        # Fully Connected Layers with Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

    def loss(self, pred, score):
        return F.mse_loss(pred, score)

In [ ]:
from sklearn.metrics import r2_score
import torch
import torch.optim as optim
from torch_geometric.data import DataLoader
from torch_geometric.data import Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import numpy as np



# Define the hyperparameter values to test
hidden_dim_values = list(range(16, 512, 16))
dropout_values = np.arange(0.05, 1.0, 0.05)
momentum_values = np.arange(0.05, 1.0, 0.05)

# Initialize a dictionary to store results
part_results ={}
results = {}

# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Loop over hyperparameters
for hidden_dim in hidden_dim_values:
    for dropout in dropout_values:
        for momentum in momentum_values:
            print(f"\nTesting for hidden_dim={hidden_dim}, dropout={dropout}, momentum={momentum}")

            # Initialize a dictionary to store results for the current hyperparameters
            hyperparam_results = {'MSE': [], 'R2': []}

            # Loop over folds
            for fold, (train_idx, val_idx) in enumerate(kf.split(train_dataset)):
                train_data = [train_dataset[i] for i in train_idx]
                val_data = [train_dataset[i] for i in val_idx]

                model = GraphRegressionModel(num_node_features=1, hidden_dim=hidden_dim, output_dim=1, dropout=dropout, momentum=momentum)
                optimizer = optim.Adam(model.parameters(), lr=0.001)

                # Training loop
                model.train()
                for epoch in tqdm(range(10), desc=f"Fold {fold + 1}/{num_folds}"):
                    for data in train_data:
                        optimizer.zero_grad()
                        output = model(data)
                        loss = model.loss(output, data.y.view(-1, 1).float())
                        loss.backward()
                        optimizer.step()

                # Validation loop
                model.eval()
                y_true = []
                y_pred = []
                
                for data in val_data:
                    with torch.no_grad():    
                        output = model(data)[0][0].item()                
                        y_pred.append(output)

                    y_true.append(data.y.item())

                # Calculate and store Mean Squared Error (MSE)
                r2 = r2_score(y_true, y_pred)
                hyperparam_results['R2'].append(r2)
                mse = mean_squared_error(y_true, y_pred)
                hyperparam_results['MSE'].append(mse)
                part_results[(hidden_dim, dropout, momentum, fold)] = {'MSE': mse, 'R2' : r2}

            # Calculate average MSE across folds
            avg_mse = sum(hyperparam_results['MSE']) / num_folds
            avg_r2 = sum(hyperparam_results['R2']) / num_folds
            results[(hidden_dim, dropout, momentum)] = {'Avg MSE': avg_mse, 'Avg R2': avg_r2}


# Open files for writing
with open('GCN_avg.txt', 'w') as file1, open('GCN_spec.txt', 'w') as file2:
    # Print the results to file1
    for (hidden_dim, dropout), metrics in results.items():
        print(f"Hidden Dim: {hidden_dim}, Dropout: {dropout}, Momentum: {momentum}, Avg MSE: {metrics['Avg MSE']}, Avg R2: {metrics['Avg R2']}", file=file1)

    # Print the partial results to file2
    for (hidden_dim, dropout, fold), metrics in part_results.items():
        print(f"Hidden Dim: {hidden_dim}, Dropout: {dropout}, Momentum: {momentum}, Fold: {fold}, MSE: {metrics['MSE']}, R2: {metrics['R2']}", file=file2)

